# 🔍 00.03: 기본 RAG 맛보기

## 💡 검색 + 생성이 어떻게 작동하는지 체험!

In [ ]:
# 필수 라이브러리 설치 (RAG 체험용)
!pip install -q langchain-community langchain-core sentence-transformers faiss-cpu transformers torch

In [ ]:
# 1. 샘플 문서 + 한국어 임베딩 벡터스토어 준비
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document

# 한국어가 섞인 간단한 문서들 (도메인 다양화)
raw_docs = [
    {
        "title": "파이썬 역사",
        "content": "파이썬은 1991년 귀도 반 로섬이 만든 고급 프로그래밍 언어이며, 현재도 활발하게 사용된다.",
        "category": "programming",
    },
    {
        "title": "머신러닝 개요",
        "content": "머신러닝은 데이터로 패턴을 학습하는 기술이며, 최근에는 딥러닝이 큰 주목을 받고 있다.",
        "category": "ai",
    },
    {
        "title": "RAG 소개",
        "content": "RAG는 검색과 생성을 결합한 시스템으로, 관련 문서를 찾아 답변의 신뢰도를 높인다.",
        "category": "ai",
    },
    {
        "title": "서울 여행",
        "content": "서울은 다양한 문화유산과 최신 트렌드가 공존하는 도시로, 주말 여행지로 인기가 높다.",
        "category": "travel",
    },
]

# LangChain Document 객체로 변환
documents = [Document(page_content=doc["content"], metadata={"title": doc["title"], "category": doc["category"]}) for doc in raw_docs]

# 한국어 임베딩 모델로 벡터스토어 구축 (bge-m3는 다국어 지원)
embedding_model = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")
vectorstore = FAISS.from_documents(documents, embedding_model)
print("✅ 벡터스토어 구축 완료")


In [ ]:
# 2. 질문으로 상위 k개 문서 검색 + 리더보드 확인
import pandas as pd

question = "RAG는 어떻게 동작해?"
results = vectorstore.similarity_search(question, k=4)

leaderboard = pd.DataFrame([
    {
        "rank": idx + 1,
        "title": res.metadata.get("title", ""),
        "category": res.metadata.get("category", ""),
        "snippet": res.page_content[:40] + "...",
    }
    for idx, res in enumerate(results)
])

print(f"🔍 질문: {question}")
display(leaderboard)


In [ ]:
# 3. 가장 관련 있는 문서 하이라이트로 확인
import re

selected_doc = results[0].page_content
keywords = question.replace('?', '').split()

highlighted = selected_doc
for kw in keywords:
    if kw:
        highlighted = re.sub(kw, f"**{kw}**", highlighted, flags=re.IGNORECASE)

print("📄 상위 문서 내용 (질문 키워드 하이라이트):")
print(highlighted)


In [ ]:
# 4. Day1에서 사용한 EXAONE instruct 모델로 답변 생성
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

def load_exaone():
    tokenizer = AutoTokenizer.from_pretrained("LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct", trust_remote_code=True)
    model = AutoModelForCausalLM.from_pretrained(
        "LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct",
        trust_remote_code=True,
        torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
        device_map="auto"
    )
    return tokenizer, model

def generate_answer(question: str, docs) -> str:
    context = " ".join([doc.page_content for doc in docs[:2]])
    prompt = f"""[SYSTEM]
아래 컨텍스트를 기반으로 질문에 한국어로 간결하고 정확하게 답변하세요.
컨텍스트에 없는 내용은 추측하지 마세요.

[CONTEXT]
{context}

[QUESTION]
{question}

[ANSWER]
"""
    inputs = exaone_tokenizer(prompt, return_tensors="pt").to(exaone_model.device)
    with torch.no_grad():
        outputs = exaone_model.generate(
            **inputs,
            max_new_tokens=120,
            temperature=0.7,
            do_sample=True,
            eos_token_id=exaone_tokenizer.eos_token_id
        )
    return exaone_tokenizer.decode(outputs[0], skip_special_tokens=True).split("[ANSWER]")[-1].strip()

exaone_tokenizer, exaone_model = load_exaone()
answer = generate_answer(question, results)
print("🤖 EXAONE 기반 답변:")
print(answer)


In [ ]:
# 5. 간단 성능 지표 보드 (키워드 커버리지 & similarity score)
from difflib import SequenceMatcher

# 키워드 커버리지(질문과 문서가 얼마나 겹치는지) 계산
# - 질문에서 중요한 키워드를 미리 정의해 두고
# - 문서에 몇 개가 포함되어 있는지 비율로 평가합니다.
def keyword_hit_ratio(text, keywords):
    return sum(1 for kw in keywords if kw.lower() in text.lower()) / len(keywords)

# Naive RAG의 한계를 보여주기 위해, 간단한 네 가지 키워드를 사용합니다.
keywords = ["RAG", "검색", "생성", "시스템"]
metrics = []

for idx, res in enumerate(results):
    # SequenceMatcher는 파이썬 표준 라이브러리에서 두 문자열의 공통 subsequence 비율을 계산합니다.
# - 텍스트를 토큰화하거나 벡터화하지 않고 문자 기반으로 비교합니다.
# - 출력은 0(완전히 다름)~1(완전히 같음) 사이의 부동소수점입니다.
# - 즉, 스파스/덴스 임베딩이 아니라 간단한 문자열 매칭을 보는 지표입니다.
# - “두 문장에 연속된 글자가 얼마나 비슷하게 나열되어 있냐”


    similarity = SequenceMatcher(None, question.lower(), res.page_content.lower()).ratio()
    metrics.append({
        "Rank": idx + 1,
        "문서 제목": res.metadata.get("title", ""),
        "카테고리": res.metadata.get("category", ""),
        "키워드 커버리지": round(keyword_hit_ratio(res.page_content, keywords), 2),
        "질문-문서 유사도": round(similarity, 2)
    })

metrics_df = pd.DataFrame(metrics)
print("📊 상위 문서 비교 (키워드 커버리지 / 질문-문서 유사도):")
display(metrics_df)

print("해석 가이드:")
print("- Rank: 검색 결과 순위 (1위가 가장 관련도가 높은 문서)")
print("- 키워드 커버리지: 질문 핵심 키워드가 문서에 얼마나 포함되어 있는지 (1.0이면 모두 포함)")
print("- 질문-문서 유사도: 전체 텍스트 기준으로 질문과 문서가 얼마나 비슷한지 (0~1 사이)")
print("➡️ 예시) 1위 문서의 커버리지/유사도가 낮다면 나이브 RAG가 텍스트 표면만 보고 잘못된 문서를 뽑았을 가능성이 큽니다.")
print("➡️ 이 지표를 바탕으로 다음 실습에서 쿼리 개선, 메타데이터 필터링 등이 왜 필요한지 연결합니다.")


In [ ]:
# 4. RAG 과정 정리
print("🎉 기본 RAG 체험 완료!")
print("✅ 1단계: 질문 입력")
print("✅ 2단계: 관련 문서 검색")
print("✅ 3단계: 컨텍스트로 답변 생성")
print("💡 다음: 00.04에서 고급 RAG 기능들 맛보기!")

## ✅ 기본 RAG 체험 정리
- 검색 상위 결과(Top-k)가 어떻게 순위와 스코어가 다른지 직접 확인
- EXAONE 모델을 사용해 한국어로 자연스러운 답변 생성
- 키워드 커버리지 / 유사도 비교를 통해 나이브 RAG의 한계(문맥 누락, 다의어 문제 등) 체감
- 다음 실습(02)에서 다루게 될 실패 패턴을 미리 힌트로 소개해 주세요.
